In [1]:
import wikipediaapi
import json
import os
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
import shutil

In [ ]:
# Define Hebrew Wikipedia API instance
def init_wikipedia(lang='he'):
    return wikipediaapi.Wikipedia(
        language=lang,
        user_agent='BenGurionBot/1.0'
    )

def extract_sections(page):
    """
    Recursively extracts all sections and subsections from a Wikipedia page.
    """
    sections_data = {}

    def recurse_sections(sections, container):
        for section in sections:
            container[section.title] = {
                "text": section.text,
                "subsections": {}
            }
            recurse_sections(section.sections, container[section.title]["subsections"])

    recurse_sections(page.sections, sections_data)
    return sections_data

def extract_wiki_page_data(page):
    """
    Extracts structured content from a Wikipedia page.
    """
    return {
        "title": page.title,
        "summary": page.summary,
        "url": page.fullurl,
        "sections": extract_sections(page)
    }

def process_wikipedia_pages_as_list(page_titles, output_path="./data/wiki_pages.json", lang='he'):
    """
    Fetches multiple Wikipedia pages and stores them as a single list of structured entries.
    Saves the full list to a single JSON file.
    """
    wiki = init_wikipedia(lang=lang)
    all_pages = []

    for title in page_titles:
        print(f"📘 Fetching '{title}'...")
        page = wiki.page(title)
        if not page.exists():
            print(f"⚠️ Page '{title}' not found.")
            continue

        data = extract_wiki_page_data(page)
        all_pages.append(data)

    # Save all data as a single JSON array
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(all_pages, f, ensure_ascii=False, indent=2)

    print(f"✅ Saved {len(all_pages)} Wikipedia entries to {output_path}")
    return all_pages  # Optional: return for immediate use

In [3]:
topics = ['דוד בן-גוריון', 'מפא"י', 'רשימת פועלי ישראל', 'ציונות', 'אחדות העבודה', 'העבודה', 'פועלי ציון']
all_data = process_wikipedia_pages_as_list(topics)

📘 Fetching 'דוד בן-גוריון'...
📘 Fetching 'מפא"י'...
📘 Fetching 'רשימת פועלי ישראל'...
📘 Fetching 'ציונות'...
📘 Fetching 'אחדות העבודה'...
📘 Fetching 'העבודה'...
📘 Fetching 'פועלי ציון'...
✅ Saved 7 Wikipedia entries to ./data/wiki_pages.json


In [4]:
print(all_data)

[{'title': 'דוד בן-גוריון', 'summary': 'דָּוִד בֶּן־גּוּרְיוֹן (גְּרין; \u200f\u200fⒾ\u200f; 16 באוקטובר 1886, י"ז בתשרי תרמ"ז – 1 בדצמבר 1973, ו\' בכסלו תשל"ד) היה מדינאי ישראלי וראש הממשלה הראשון של מדינת ישראל.\nבן־גוריון היה איש העלייה השנייה, ממנהיגי תנועת העבודה, חבר במפלגות פועלי ציון ואחדות העבודה, מזכירה הכללי השני של ההסתדרות הכללית של העובדים בארץ ישראל ומנהיגה הראשון של מפלגת מפא"י.\nבתפקידיו כיושב ראש הנהלת הסוכנות היהודית ויושב ראש ההנהלה הציונית, עמד בראש הנהגת היישוב העברי בארץ ישראל בתקופת המנדט הבריטי, ובתפקידו כיושב ראש מנהלת העם מילא תפקיד מרכזי בניסוח מגילת העצמאות, היה הראשון לחתום עליה והכריז ב־14 במאי 1948 על הקמת מדינת ישראל. עם הקמת המדינה החל לפעול למען הממלכתיות, ובזמן מלחמת העצמאות החליט על פירוק המחתרות, וחתם על הפקודה להקמת צה"ל. על מנת לקיים צבא ממלכתי אחד, הורה על הפגזת האונייה אלטלנה ועל פירוק מטה הפלמ"ח.\nהוא הנהיג את מדינת ישראל כראש הממשלה וכשר הביטחון במשך עשור וחצי עד 1963 (מלבד פסק זמן בן כשנתיים, בשנים 1953–1955). כראש הממשלה הראשון היה אחראי במ

In [7]:
def split_text(text, max_words=200, overlap=20):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+max_words]
        chunks.append(" ".join(chunk))
        i += max_words - overlap
    return chunks

def wiki_json_to_chunks(json_path, source_id="wikipedia_json", max_words=260, overlap=35, start_idx=0):
    """
    Loads one or more Wikipedia pages from a JSON file and returns a list of Document chunks.
    Each chunk has metadata including source, section title, and a running index.
    """
    with open(json_path, encoding='utf-8') as f:
        wiki_data = json.load(f)

    # Handle both a single dict or a list of dicts
    pages = wiki_data if isinstance(wiki_data, list) else [wiki_data]

    docs = []
    current_idx = start_idx

    def recurse_sections(sections, page_title, parent_title=None):
        nonlocal current_idx
        for title, content in sections.items():
            full_title = f"{parent_title} > {title}" if parent_title else title
            text = content.get("text", "").strip()
            chunks = split_text(text, max_words, overlap)
            for chunk in chunks:
                docs.append(Document(
                    page_content=chunk,
                    metadata={
                        "source": source_id,
                        "book_id": page_title,
                        "section": full_title,
                        "idx": current_idx
                    }
                ))
                current_idx += 1
            recurse_sections(content.get("subsections", {}), page_title, full_title)

    for page in pages:
        recurse_sections(page["sections"], page["title"])

    return docs

In [8]:
import pickle

with open('./data/combined_chunks.pkl', 'rb') as f:
    chunks = pickle.load(f)

max_existing_idx = max(doc.metadata["idx"] for doc in chunks)

wiki_docs = wiki_json_to_chunks(
    json_path="./data/wiki_pages.json",
    start_idx=max_existing_idx + 1
)

In [10]:
len(wiki_docs)

291

In [ ]:
api_key = "your-api-key"
os.environ["OPENAI_API_KEY"] = api_key

In [ ]:
def create_augmented_vectorstore_from_json(
    original_index_path,
    output_index_path,
    wiki_chunks,
    embedding_model=OpenAIEmbeddings(model="text-embedding-3-large")
):
    # 1. Copy the original index directory
    if os.path.exists(output_index_path):
        raise FileExistsError(f"{output_index_path} already exists. Choose a new path.")
    shutil.copytree(original_index_path, output_index_path)

    # 2. Load the copied index as a new vectorstore
    vectorstore = FAISS.load_local(output_index_path, embeddings=embedding_model, allow_dangerous_deserialization=True)

    # 3. Add new documents to the vectorstore
    vectorstore.add_documents(wiki_chunks)
    vectorstore.save_local(output_index_path)

    print(f"✅ Augmented vectorstore created at {output_index_path} with {len(wiki_chunks)} wiki chunks.")


In [14]:
create_augmented_vectorstore_from_json("./faiss_index_openai_3textlarge_full", "./faiss_index_openai_3textlarge_full_copy", wiki_docs)

✅ Augmented vectorstore created at ./faiss_index_openai_3textlarge_full_copy with 291 wiki chunks.


In [ ]:
combined_chunks = chunks + wiki_docs

with open('./data/combined_chunks.pkl', 'wb') as f:
    pickle.dump(combined_chunks, f)